## Configuración general

Esto es configuración general de IPython

In [1]:
%config InlineBackend.close_figures=False
%matplotlib inline
from matplotlib import interactive
interactive(False)

Luego importamos todo las "clases" que provee la librería `sur`

In [2]:
from sur import *
m = Mixture()

## Definición de la mezcla

In [3]:
mi_mezcla = Mixture(name="tal cosa")
mi_mezcla.add('methane', .4) 
mi_mezcla['ethane'] = .3
mi_mezcla['co2'] = .2002
mi_mezcla.add('water')
mi_mezcla.compounds


[<Compound: METHANE>, <Compound: ETHANE>, <Compound: CARBON DIOXIDE>, <Compound: WATER>]

In [4]:
mi_mezcla.sort(True)   # ordena por peso

In [5]:
mi_mezcla.compounds

[<Compound: METHANE>, <Compound: WATER>, <Compound: ETHANE>, <Compound: CARBON DIOXIDE>]

## Definición de la configuración (RKPR)

In [6]:
setup = EosSetup.objects.create(eos='PR', kij_mode=EosSetup.CONSTANTS,  lij_mode=EosSetup.ZERO)
setup.set_interaction('kij', 'methane', 'co2', .1)
setup.set_interaction('kij', 'co2', 'n-decane',  0.091)
setup.set_interaction('lij', 'co2', 'n-decane',  -0.90)



<LijInteractionParameter: PR [<Compound: n-DECANE>, <Compound: CARBON DIOXIDE>]: -0.9>

In [7]:
matriz = """0. .2 .3 .4
0 0 -.1 .2
0 0 -.3 .12
0 0 -.12 .21"""


Podemos ver como van quedando nuestras matrices de interacción para la mezcla `m`

In [8]:
setup.set_interaction_matrix('kij', mi_mezcla, matriz)

In [9]:
setup.kij(mi_mezcla)

array([[ 0.  ,  0.2 ,  0.3 ,  0.4 ],
       [ 0.2 ,  0.  , -0.1 ,  0.2 ],
       [ 0.3 , -0.1 ,  0.  ,  0.12],
       [ 0.4 ,  0.2 ,  0.12,  0.  ]])

## Simulación de la envolvente para la mezcla con la configuración dada

Para simular una envolvente se llama al método `get_envelope` de la mezcla con una configuración

In [10]:
env = mi_mezcla.get_envelope(setup)


  warnings.warn('loadtxt: Empty input file: "%s"' % fname)



UnboundLocalError: local variable 'end' referenced before assignment

en `env` (variable) tenemos la envolvente. Podemos consultar, por ejemplo, sus atributos temperatura, presión, etc.





In [ ]:
env.p

## Graficación de envolventes

In [ ]:
fig = env.plot()
fig

In [ ]:
multiplot??

La figura puede manipularse a posteriori. Por ejemplo

In [ ]:
ax = fig.get_axes()[0]
ax.set_yticks([0] + ax.get_yticks()[1:])
fig

In [ ]:
ax.set_ylim((-50/3, 250))
fig

Pero también, acepta algunos parámetros de configuración en la llamada. Por ejemplo, el parámetro `format` permite definir el estilo

In [ ]:
env.plot(format='k')   # envolvente en linea negra continua

In [ ]:
env.plot(critical_point='^', format='|y')    # envolvente en amarillo punteado y punto critico como un triangulo

## Datos experimentales

Se pueden asociar datos experimentales de una envolvente a una mezcla a traves de `experimental_envelope`. Por ejemplo, a modo de ejemplo, alteramos los datos de la envolvente calculada a modo de datos experimentales.

In [ ]:
n = env.p.size

p = env.p[np.arange(0, n, 7)]  * 1.1
t = env.t[np.arange(0, n, 7)]
exp_env = m.experimental_envelope(t, p)
exp_fig = exp_env.plot()
exp_fig

Los graficos pueden encadenarse. Por ejemplo, podemos graficar la envolvente simulada y nuestra envolvente *experimental*

In [ ]:
env.plot(exp_fig)

## Cálculo de flashes

Un flash se representa como un objeto que produce dos mezclas con los mismos compuestos que la mezcla de entrada pero distinta fraccion. 

In [ ]:
flash = m.get_flash(setup, 400, 100)

In [ ]:
flash.vapour_mixture

In [ ]:
flash.liquid_mixture.z

O, lo que es lo mismo

In [ ]:
flash.x

`liquid_mixture` y `vapour_mixture` son mezclas, por lo que podemos simular sus envolventes con el mismo setup. Luego, grafiquemos todo junto

In [ ]:
le = flash.liquid_mixture.get_envelope(setup)
ve = flash.vapour_mixture.get_envelope(setup)

env.plot(ve.plot(le.plot()))

In [ ]:
fig = env.plot()

In [ ]:
fig.